# 05 — Capacity Ceiling Smoothing

Illustrates the stabilizer concept: feed SD/trade a smoothed ceiling rather than raw capacities.

In [ ]:
from pathlib import Path
import pandas as pd
import yaml
import matplotlib.pyplot as plt

def find_repo_root(start: Path) -> Path:
    for p in [start, *start.parents]:
        if (p/"configs"/"time.yml").exists() and (p/"data"/"exogenous").exists():
            return p
    raise FileNotFoundError("Could not locate repo root containing configs/ and data/exogenous/")

ROOT = find_repo_root(Path(".").resolve())
CFG = ROOT/"configs"
DATA = ROOT/"data"/"exogenous"

def load_yaml(path):
    with open(path, "r", encoding="utf-8") as f:
        return yaml.safe_load(f)


time_cfg = load_yaml(CFG/'time.yml')
alpha = time_cfg.get('coupling',{}).get('stabilizer',{}).get('lambda_0_1', 0.25)
cap = pd.read_csv(DATA/'capacity_stage_observed_kt_per_yr.csv')

REGION='EU27'; MAT='Zn'; STAGE='refining'
series = cap[(cap['r']==REGION) & (cap['m']==MAT) & (cap['p']==STAGE)].copy()
series = series[['t','value']].sort_values('t')

# Exponential smoothing
y = series['value'].to_numpy()
ys = y.copy()
for i in range(1,len(y)):
    if pd.isna(y[i]):
        ys[i] = ys[i-1]
    elif pd.isna(ys[i-1]):
        ys[i] = y[i]
    else:
        ys[i] = alpha*y[i] + (1-alpha)*ys[i-1]

plt.figure();
plt.plot(series['t'], y, label='raw')
plt.plot(series['t'], ys, label='smoothed')
plt.title(f'Capacity ceiling — raw vs smoothed (alpha={alpha})\n{REGION} {MAT} {STAGE}')
plt.xlabel('Year'); plt.ylabel('kt/yr')
plt.legend();
plt.show()
